# Fome Zero Foods - Análise de dados

Arquivo para limpeza, tratamento de dados e criação de um dataset limpo.

## 1. Bibliotecas

In [1]:
import inflection
import pandas as pd

from pathlib import Path

## 2. Importação dos dados

### 2.1 Importando o dataset bruto

In [2]:
# caminho do arquivo
path = Path.cwd().parent / 'data' / 'raw' / 'dataset.csv'

try:
    with open(path, 'r', encoding='utf-8') as arquivo:
        df_raw = pd.read_csv(arquivo)
except FileNotFoundError:
    print(f"Arquivo não encontrado: {path}")


In [3]:
df_raw.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",...,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


### 2.2 Verificando informações dos dados

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7527 entries, 0 to 7526
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         7527 non-null   int64  
 1   Restaurant Name       7527 non-null   object 
 2   Country Code          7527 non-null   int64  
 3   City                  7527 non-null   object 
 4   Address               7527 non-null   object 
 5   Locality              7527 non-null   object 
 6   Locality Verbose      7527 non-null   object 
 7   Longitude             7527 non-null   float64
 8   Latitude              7527 non-null   float64
 9   Cuisines              7512 non-null   object 
 10  Average Cost for two  7527 non-null   int64  
 11  Currency              7527 non-null   object 
 12  Has Table booking     7527 non-null   int64  
 13  Has Online delivery   7527 non-null   int64  
 14  Is delivering now     7527 non-null   int64  
 15  Switch to order menu 

In [5]:
df_raw.isnull().sum()

Restaurant ID            0
Restaurant Name          0
Country Code             0
City                     0
Address                  0
Locality                 0
Locality Verbose         0
Longitude                0
Latitude                 0
Cuisines                15
Average Cost for two     0
Currency                 0
Has Table booking        0
Has Online delivery      0
Is delivering now        0
Switch to order menu     0
Price range              0
Aggregate rating         0
Rating color             0
Rating text              0
Votes                    0
dtype: int64

In [6]:
print("Valores únicos das colunas:")
df_raw.nunique()

Valores únicos das colunas:


Restaurant ID           6942
Restaurant Name         5914
Country Code              15
City                     125
Address                 6760
Locality                2272
Locality Verbose        2357
Longitude               6846
Latitude                6833
Cuisines                2832
Average Cost for two     156
Currency                  12
Has Table booking          2
Has Online delivery        2
Is delivering now          2
Switch to order menu       1
Price range                4
Aggregate rating          30
Rating color               7
Rating text               28
Votes                   1739
dtype: int64

## 3. Limpeza dos dados

### 3.1 Definindo funções

Aqui temos as funções que serão utilizadas na limpeza do dataframe

- 1. country_name()
- 2. create_price_tye()
- 3. color_name()
- 4. rename_columns()

In [11]:

def country_name(country_id: int) -> str:
    """
    Função para converter o código do país no nome do país.

    Args:
        country_id (int): Número que representa do código do país

    Returns:
        str: String com o nome do país.

    Examples:
        print(country_name(30))
        Output: 'Brazil'
    """

    COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
    }

    return COUNTRIES[country_id]

def create_price_tye(price_range: int) -> str:
    """
    Função para converter um código no tipo de preço da comida.

    Args:
        price_range (int): Número de 1 a 4 que representa o tipo de preço.

    Returns:
        str: String com o tipo de preço

    Examples:
        print(create_price_tye(2))
        Output: 'normal'
    """
    
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

def color_name(color_code: str) -> str:
    """
    Função para converter o código de cor para nome da cor.

    Args:
        color_code (str): String com o código da cor a ser convertido

    Returns:
        str: String com a cor convertida

    Example:
        print(color_name("FFBA00"))
        Output: 'red'
    """
    COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
    }

    return COLORS[color_code]

def rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Função para renomear colunas do Dataframe

    Args:
        df (pd.DataFrame): Dataframe de entrada.

    Returns:
        pd.DataFrame: Dataframe com colunas renomeadas.
    
    Example:
        df_rename = rename_columns(df)
    """

    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new

    return df

def clean_cuisines_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Função para limpar e excluir as linhas vazias da coluna 'cuisines'
    Etapas:
        1. Limpeza a padronização dos nomes dos nomes do tipos de cozinhas, 
        pegando apenas o primeiro nome da String
        2. Remoção das linhas vazias

    Args:
        df (pd.DataFrame): Dataframe de entrada contendo a coluna 'cuisines'

    Returns:
        pd.DataFrame: Dataframe com a coluna limpa
    """

    # Pega apenas o primeiro item da lista de culinárias
    df['cuisines'] = df['cuisines'].str.split(',').str[0]

    # Remove as linhas vazias
    df = df.dropna(subset=['cuisines'])

    return df

def clean_rating_text(text: str) -> str | None:
    """
    Função para limpar a coluna 'rating_text' e padronizar suas notas, com nomes
    no idioma Inglês.

    Args:
        text (str): String de entreda com categoria da nota para padronização

    Returns:
        str | None: Retorna o valor padronizado.
    """

    # Criando um dicionário com a tradução/padronização
    mapping = {
        # Categoria: Excellent
        'Excellent': 'Excellent',
        'Excelente': 'Excellent',
        'Eccellente': 'Excellent', # Italiano
        'Vynikajúce': 'Excellent', # Eslovaco
        'Skvělá volba': 'Excellent', # Tcheco (Ótima escolha)
        'Skvělé': 'Excellent',       # Tcheco
        'Wybitnie': 'Excellent',     # Polonês (Excepcional)
        'Harika': 'Excellent',       # Turco (Maravilhoso)
        'Terbaik': 'Excellent',      # Indonésio (O melhor)

        # Categoria: Very Good
        'Very Good': 'Very Good',
        'Muito bom': 'Very Good',
        'Muito Bom': 'Very Good',    # Variação de caixa
        'Bardzo dobrze': 'Very Good',# Polonês
        'Muy Bueno': 'Very Good',    # Espanhol
        'Velmi dobré': 'Very Good',  # Tcheco
        'Veľmi dobré': 'Very Good',  # Eslovaco
        'Çok iyi': 'Very Good',      # Turco
        'Sangat Baik': 'Very Good',  # Indonésio

        # Categoria: Good
        'Good': 'Good',
        'Bueno': 'Good',             # Espanhol
        'Bom': 'Good',               # Português
        'Buono': 'Good',             # Italiano
        'Baik': 'Good',              # Indonésio
        'İyi': 'Good',               # Turco

        # Categoria: Average
        'Average': 'Average',
        'Biasa': 'Average',          # Indonésio (Comum/Médio)

        # Categoria: Poor
        'Poor': 'Poor',

        # Categoria: Not Rated
        'Not rated': 'Not rated'
    }

    # Retorna o valor padronizado.
    # O comando .get(text, text) tenta achar no dicionário;
    # se não achar, mantém o texto original (segurança).
    return mapping.get(text, text)


### 3.2 Execução da Limpeza

In [12]:
# Cópia do dataframe para limpeza
df = df_raw.copy()

# Elimininando coluna 'Switch to order menu' contento apenas um valor único
df = df.drop(columns=['Switch to order menu'])

# Renomeia colunas
df = rename_columns(df)

# Criando coluna com nome do país
df['country_name'] = df['country_code'].apply(lambda x: country_name(x))

# Criando coluna com código do preço da comida
df['price_type'] = df['price_range'].apply(lambda x: create_price_tye(x))

# Criando coluna com nome da cor
df['color_name'] = df['rating_color'].apply(lambda x: color_name(x))

# Limpeza e remoção de linhas vazias da coluna 'cuisines'
df = clean_cuisines_column(df)

# Renomeado as linhas da coluna 'rating_text' de forma padronizada
df['rating_text'] = df['rating_text'].apply(clean_rating_text)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7512 entries, 0 to 7526
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   restaurant_id         7512 non-null   int64  
 1   restaurant_name       7512 non-null   object 
 2   country_code          7512 non-null   int64  
 3   city                  7512 non-null   object 
 4   address               7512 non-null   object 
 5   locality              7512 non-null   object 
 6   locality_verbose      7512 non-null   object 
 7   longitude             7512 non-null   float64
 8   latitude              7512 non-null   float64
 9   cuisines              7512 non-null   object 
 10  average_cost_for_two  7512 non-null   int64  
 11  currency              7512 non-null   object 
 12  has_table_booking     7512 non-null   int64  
 13  has_online_delivery   7512 non-null   int64  
 14  is_delivering_now     7512 non-null   int64  
 15  price_range           7512

###  3.3 Salvando o dataframe limpo

In [14]:
# Salvando o dataframe limpo no diretório data/processed/
saved_path = Path.cwd().parent/'data'/'processed'/'dataset_processed.csv'

df.to_csv(saved_path, index=False)